In [1]:
import tensorflow as tf 
import tensorflow_datasets as tfds
import numpy as np 
import jax 
import jax.numpy as jnp

# change the backend of keras to jax
import os
os.environ["KERAS_BACKEND"] = "jax"

import keras_core as kc

2023-09-17 16:38:34.023518: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-17 16:38:34.068904: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using JAX backend.


In [2]:
ds = tfds.load('mnist', split="train", shuffle_files=True)
ds = ds.shuffle(1024).batch(32).prefetch(tf.data.AUTOTUNE)
dsIter = iter(ds)

2023-09-17 16:38:37.087716: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-17 16:38:37.095502: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-17 16:38:37.096567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-17 16:38:37.100871: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-17 16:38:37.102522: I tensorflow/compile

In [3]:

def cnn():
    x = kc.layers.Input([28, 28, 1])
    c1 = kc.layers.Conv2D(32, (3,3), padding="same", activation=kc.activations.elu)(x)
    c2 = kc.layers.Conv2D(32, (3,3), padding="same", activation=kc.activations.elu)(c1)
    
    flatten = kc.layers.Flatten()(c2)
    
    fc1 = kc.layers.Dense(512, kc.activations.mish)(flatten)
    fc2 = kc.layers.Dense(256, kc.activations.mish)(fc1)
    out = kc.layers.Dense(10, kc.activations.mish)(fc2)
    
    return kc.Model(inputs=x, outputs=out)

model = cnn()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)        │        320 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 32)        │      9,248 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 25088)             │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 512)               │ 12,845,568 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 256)               │    131,328 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_2 (Dense)                 │ (None, 10)                │      2,570 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 12,989,034 (49.55 MB)

 Trainable params: 12,989,034 (49.55 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
dsFetcher = next(dsIter)
img = (tf.cast(dsFetcher['image'], tf.float32) - 128)/128
model(img)

Array([[ 6.22420199e-02, -2.80661676e-02, -3.74503136e-02,
         8.35650638e-02, -8.57112706e-02,  5.02679572e-02,
        -1.15435133e-02, -3.29992697e-02,  2.10827086e-02,
         1.19653381e-01],
       [-1.75538636e-03, -9.19433981e-02,  1.58181619e-02,
         4.24447656e-02, -1.12760969e-01,  3.81765403e-02,
         3.49016897e-02, -5.32594323e-02, -8.14591348e-02,
         9.55777690e-02],
       [ 5.65132909e-02, -7.18830600e-02, -1.47164175e-02,
         8.59369785e-02, -1.45835981e-01,  6.08097725e-02,
         1.88168064e-01, -2.00289413e-02,  4.88493033e-02,
         2.63666391e-01],
       [ 1.13367341e-01, -1.26303528e-02, -5.14497273e-02,
         6.03842735e-02, -9.06155109e-02, -5.33444732e-02,
         2.02173889e-02, -3.34802680e-02,  8.86093602e-02,
         1.96196184e-01],
       [-9.51141026e-03, -1.24743190e-02,  4.45703864e-02,
         1.11030536e-02,  2.35731807e-02,  5.47692329e-02,
         3.92927136e-03, -1.07513264e-01, -1.24888323e-01,
         1.